# Bin packing

<div class="alert alert-block alert-info">
    &#9432; The code in this notebook can be executed <a href="https://www.opvious.io/notebooks/retro/notebooks/?path=examples/bin-packing.ipynb">directly from your browser</a>.
</div>

This notebook contains a mixed-integer program implementation of the canonical [bin packing problem](https://en.wikipedia.org/wiki/Bin_packing_problem).

In [1]:
%pip install opvious

## Formulation

The first step is to formulate the problem problem using `opvious`' [declarative modeling API](https://opvious.readthedocs.io/en/stable/modeling.html).

In [2]:
import opvious.modeling as om

class BinPacking(om.Model):
    """Bin-packing MIP formulation"""
    
    items = om.Dimension() # Set of items to be put into bins
    weight = om.Parameter.non_negative(items) # Weight of each item
    bins = om.interval(1, om.size(items), name="B") # Set of bins
    max_weight = om.Parameter.non_negative() # Maximum weight allowed in a bin
    assigned = om.Variable.indicator(bins, items, qualifiers=['bins']) # 1 if an item is assigned to a given bin, 0 otherwise
    used = om.Variable.indicator(bins) # 1 if a bin is used, 0 otherwise

    @om.constraint
    def each_item_is_assigned_once(self):
        """Constrains each item to be assigned to exactly one bin"""
        for i in self.items:
            yield om.total(self.assigned(b, i) for b in self.bins) == 1

    @om.constraint
    def bin_weights_are_below_max(self):
        """Constrains each bin's total weight to be below the maximum allowed"""
        for b in self.bins:
            bin_weight = om.total(self.weight(i) * self.assigned(b, i) for i in self.items)
            yield bin_weight <= self.used(b) * self.max_weight()

    @om.objective
    def minimize_bins_used(self):
        """Minimizes the total number of bins with at least one item"""
        return om.total(self.used(b) for b in self.bins)

model = BinPacking()

We can view its mathematical definitions by printing its `specification`.

In [3]:
model.specification()

<div style="margin-top: 1em; margin-bottom: 1em;">
<details open>
<summary style="cursor: pointer; text-decoration: underline; text-decoration-style: dotted;">BinPacking</summary>
<div style="margin-top: 1em;">
$$
\begin{align*}
  \S^d_\mathrm{items}&: I \\
  \S^p_\mathrm{weight}&: w \in \mathbb{R}_+^{I} \\
  \S^a&: B \doteq \{ 1 \ldots \# I \} \\
  \S^p_\mathrm{maxWeight}&: w^\mathrm{max} \in \mathbb{R}_+ \\
  \S^v_\mathrm{assigned[bins]}&: \alpha \in \{0, 1\}^{B \times I} \\
  \S^v_\mathrm{used}&: \psi \in \{0, 1\}^{B} \\
  \S^c_\mathrm{eachItemIsAssignedOnce}&: \forall i \in I, \sum_{b \in B} \alpha_{b,i} = 1 \\
  \S^c_\mathrm{binWeightsAreBelowMax}&: \forall b \in B, \sum_{i \in I} w_{i} \alpha_{b,i} \leq \psi_{b} w^\mathrm{max} \\
  \S^o_\mathrm{minimizeBinsUsed}&: \min \sum_{b \in B} \psi_{b} \\
\end{align*}
$$
</div>
</details>
</div>

## Application

Now that we have formulated the problem, it takes just a few lines to get solutions. Since all solves run remotely--no local solver installation required--we can run it from any browser.

In [4]:
import opvious

async def optimal_assignment(bin_max_weight, item_weights):
    """Returns a grouping of items which minimizes the number of bins used
    
    Args:
        bin_max_weight: The maximum allowable total weight for all items assigned to a given bin
        item_weights: Mapping from item name to its (non-negative) weight
    """
    problem = opvious.Problem(
        specification=model.specification(),
        parameters={'weight': item_weights, 'maxWeight': bin_max_weight},
    )
    client = opvious.Client.from_environment(default_endpoint=opvious.DEMO_ENDPOINT)
    solution = await client.solve(problem)
    assignment = solution.outputs.variable('assigned')
    return list(assignment.reset_index().groupby('bins')['items'].agg(tuple))

Let's try our implementation on a simple example with 3 items.

In [5]:
await optimal_assignment(15, {
    'light': 5,
    'medium': 10,
    'heavy': 15,
})

[('heavy',), ('light', 'medium')]

That's it for this example! From here, you might be interested in browsing our [other notebooks](https://www.opvious.io/notebooks/retro).